# Task 1. Producing the data

In [ ]:
# import library
from time import sleep
from json import dumps
from kafka import KafkaProducer
import datetime as dt
import pandas as pd

In [ ]:
def publish_message(producer_instance, topic_name, data):
    try:
        # Send the data to kafka
        producer_instance.send(topic_name, data)
        # Display the data
        print('Message published successfully. Data: ' + str(data))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))
        
def connect_kafka_producer():
    _producer = None
    try:
        # create producer
        _producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                                  value_serializer=lambda x: dumps(x).encode('ascii'),
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer



if __name__ == '__main__':
    # Identify the topic name
    topic = 'December_count'
    # Create a producer
    producer = connect_kafka_producer()
    # Read the csv to pandas df
    df = pd.read_csv('Streaming_Pedestrian_December_counts_per_hour.csv')
    while True:
        # The for loop is to find each day row.
        for x in range(1,32):
            # day is a list to store rows in each day
            day = df[(df['Mdate'] == x)]
            for y in range(day.shape[0]):
                # Use dictionary to store each row data
                data = {'ID':str(day.iloc[y,0]),'Date_Time':str(day.iloc[y,1]),'Year':str(day.iloc[y,2]),'Month':str(day.iloc[y,3]),'Mdate':str(day.iloc[y,4]), 'Day':str(day.iloc[y,5]),'Time':str(day.iloc[y,6]),'Sensor_ID':str(day.iloc[y,7]),'Sensor_Name':str(day.iloc[y,8]),'Hourly_Counts':str(day.iloc[y,9])}
                publish_message(producer, topic, data)
            sleep(5)
        break